In [1]:
!wget https://vision.cs.hacettepe.edu.tr/files/recipeqa/train.json
!wget https://vision.cs.hacettepe.edu.tr/files/recipeqa/val.json
!wget https://vision.cs.hacettepe.edu.tr/files/recipeqa/test.json

--2021-01-03 04:38:26--  https://vision.cs.hacettepe.edu.tr/files/recipeqa/train.json
Resolving vision.cs.hacettepe.edu.tr (vision.cs.hacettepe.edu.tr)... 193.140.236.29
Connecting to vision.cs.hacettepe.edu.tr (vision.cs.hacettepe.edu.tr)|193.140.236.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158656297 (151M) [application/json]
Saving to: ‘train.json’

train.json          100%[===================>] 151.31M  13.8MB/s    in 14s     

2021-01-03 04:38:43 (10.9 MB/s) - ‘train.json’ saved [158656297/158656297]

--2021-01-03 04:38:43--  https://vision.cs.hacettepe.edu.tr/files/recipeqa/val.json
Resolving vision.cs.hacettepe.edu.tr (vision.cs.hacettepe.edu.tr)... 193.140.236.29
Connecting to vision.cs.hacettepe.edu.tr (vision.cs.hacettepe.edu.tr)|193.140.236.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19501603 (19M) [application/json]
Saving to: ‘val.json’

val.json            100%[===================>]  18.60M  5.60MB/s    i

In [2]:
import json
import pandas as pd

In [3]:
def process_file(file:str)->pd.DataFrame:
    with open(file,'r') as f:
        file_json = json.load(f)
        file_data = [{'body':d['body'],
            'title':d['title']
            } for b in file_json['data'] for d in b['context'] if d.get('title')]
        file_df = pd.DataFrame(file_data)
        return file_df

In [59]:
train_df = process_file('train.json')
val_df = process_file('val.json')
test_df = process_file('test.json')

In [60]:
train_df['title'].value_counts()

Ingredients               5641
Enjoy!                    1558
Bake                       914
Serve                      614
Enjoy                      557
                          ... 
Bonus Note                   1
Cover With Cheese            1
Assemble the Spices          1
Coagulation                  1
Get That Yeast Working       1
Name: title, Length: 31473, dtype: int64

#### Plans? We Don't Need No Stinkin' Plans!

In [61]:
train_df[train_df['title']=="Plans? We Don't Need No Stinkin' Plans!"]['body'].values[0]

"But I would strongly suggest you at least have some sort of idea of how you want to decorate this cake. I didn't and just made it up as I went. Doodle out a few ideas of how you want it to look (if you want. It's not required, and I'm not your mother. Have fun with it!) Once you have an idea in mind, begin kneading and softening your fondant to roll it out. If you want grey fondant, use a LOT of white, and a pinch of black and begin kneading and mixing. If you have never worked with fondant before, you can find any number of tutorials on YouTube or cake websites or blogs. It isn't hard, but it does take a lot of work (and forearm strength) to mix colors and get it all rolled out. You can also use your fondant colors to color the fondant the way you want. I stuck with a flat grey all over, and then went back and painted each piece, to save on color-mixing effort. Once you have your fondant rolled out, remove the cake from the freezer (be sure to remove the toothpicks or skewers if you 

# Deleting Duplicates

In [62]:
train_df = train_df.drop_duplicates(['body'])
val_df = val_df.drop_duplicates(['body'])
test_df = test_df.drop_duplicates(['body'])

In [63]:
# train_df = train_df.append(val_df)

In [64]:
train_df.shape[0], val_df.shape[0], test_df.shape[0]

(47704, 5795, 5863)

In [65]:
train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
test_df.to_csv('test.csv')

In [66]:
train_df.head()

,body,title
0,3 until 5 whole vanilla beans250 gram of veget...,Ingredients Halal Vanilla Extract
1,Scrape Vanilla Beans and get the seeds into ve...,Scrape Vanilla Beans
2,Vanilla Beans Seed and Vegetable Glycerin,Vegetable Glycerin and Vanilla Beans
3,Whole Vanilla Beans put in a bottle with seeds...,Vanilla Beans Can Use With Vegetable Glycerin
4,Ingredients:-250 g dark chocolate;-2 dl wippin...,WHAT YOU NEED


# Modelling

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator
from torchtext import data

import spacy
import numpy as np

import random
import math
import time

In [68]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [69]:
spacy_en = spacy.load('en')

In [70]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [71]:
SRC = Field(tokenize= tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [72]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [73]:
fields = [('Unnamed: 0', None), ('body', SRC), ('title', TRG)]

In [74]:
train_data, valid_data, test_data = data.TabularDataset.splits(format='csv',
                           path = './',
                           skip_header=True,
                           train='train.csv',
                           validation='val.csv',
                           test='test.csv',
                           fields=fields)

In [75]:
len(train_data), len(valid_data), len(test_data)

(47704, 5795, 5863)

In [76]:
print(vars(train_data.examples[100]))

{'body': ['in', 'another', 'small', 'bowl', ',', 'combine', 'the', 'soy', 'sauce', ',', 'sake', ',', 'rice', 'wine', 'vinegar', ',', 'brown', 'sugar', ',', 'msg', ',', 'and', '1/4', 'tsp', 'salt', 'for', 'the', 'noodle', 'sauce', 'part', '2', '.'], 'title': ['prepare', 'noodle', 'sauce', 'part', '2']}


In [77]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [78]:
BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort=False, 
    device = device)

In [79]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [80]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [81]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [82]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [83]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(24920, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(4274, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=4274, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [84]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,100,530 trainable parameters


In [85]:
optimizer = optim.Adam(model.parameters())

In [86]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [87]:
from tqdm.auto import tqdm

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in tqdm(enumerate(iterator)):
        
        src = batch.body
        trg = batch.title
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [88]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.body
            trg = batch.title

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [89]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [90]:
# for batch in train_iterator:
#     print(dir(batch))
#     break

In [91]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


Epoch: 01 | Time: 1m 38s
	Train Loss: 4.915 | Train PPL: 136.352
	 Val. Loss: 4.798 |  Val. PPL: 121.266



Epoch: 02 | Time: 1m 39s
	Train Loss: 4.600 | Train PPL:  99.531
	 Val. Loss: 4.786 |  Val. PPL: 119.802



Epoch: 03 | Time: 1m 40s
	Train Loss: 4.507 | Train PPL:  90.657
	 Val. Loss: 4.792 |  Val. PPL: 120.523



Epoch: 04 | Time: 1m 39s
	Train Loss: 4.442 | Train PPL:  84.955
	 Val. Loss: 4.783 |  Val. PPL: 119.444



Epoch: 05 | Time: 1m 42s
	Train Loss: 4.367 | Train PPL:  78.830
	 Val. Loss: 4.796 |  Val. PPL: 121.021



Epoch: 06 | Time: 1m 45s
	Train Loss: 4.313 | Train PPL:  74.645
	 Val. Loss: 4.791 |  Val. PPL: 120.400



Epoch: 07 | Time: 1m 46s
	Train Loss: 4.292 | Train PPL:  73.131
	 Val. Loss: 4.796 |  Val. PPL: 121.044



Epoch: 08 | Time: 1m 45s
	Train Loss: 4.266 | Train PPL:  71.216
	 Val. Loss: 4.797 |  Val. PPL: 121.144



Epoch: 09 | Time: 1m 49s
	Train Loss: 4.337 | Train PPL:  76.450
	 Val. Loss: 4.806 |  Val. PPL: 122.258



Epoch: 10 | Time: 1m 44s
	Train Loss: 4.160 | Train PPL:  64.049
	 Val. Loss: 4.798 |  Val. PPL: 121.287


In [92]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.775 | Test PPL: 118.540 |
